《PyTorch 实战》第六章习题2，加载葡萄酒数据集，并使用适当数量的输入参数创建新的模型

导入库

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

导入葡萄酒数据

In [2]:
data = np.loadtxt("data/p1ch4/tabular-wine/winequality-white.csv",
                  skiprows=1,
                  delimiter=";",
                  dtype=np.float32)
data

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

拆分出最后一列作为目标结果

In [3]:
src_data = data[:, :-1]
target_data = np.array(data[:, -1],dtype=int)
src_data, target_data

(array([[ 7.  ,  0.27,  0.36, ...,  3.  ,  0.45,  8.8 ],
        [ 6.3 ,  0.3 ,  0.34, ...,  3.3 ,  0.49,  9.5 ],
        [ 8.1 ,  0.28,  0.4 , ...,  3.26,  0.44, 10.1 ],
        ...,
        [ 6.5 ,  0.24,  0.19, ...,  2.99,  0.46,  9.4 ],
        [ 5.5 ,  0.29,  0.3 , ...,  3.34,  0.38, 12.8 ],
        [ 6.  ,  0.21,  0.38, ...,  3.26,  0.32, 11.8 ]], dtype=float32),
 array([6, 6, 6, ..., 6, 7, 6]))

加载到 `pytorch` 张量

In [4]:
src_data = torch.from_numpy(src_data)
target_data = torch.from_numpy(target_data)
src_data, target_data

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 tensor([6, 6, 6,  ..., 6, 7, 6]))

对目标进行独热编码，需要先对 `target_data` 升维

In [5]:
target_data = target_data.unsqueeze(1)
target_data

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [6]:
onehot_target = torch.zeros(target_data.shape[0],10)
onehot_target.scatter_(1,target_data,1.0)
onehot_target

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

`scatter_(dim, index, src, reduce=None)` 函数执行逻辑：
> self[index[ i ] [ j ] [ k ]] [ j ] [ k ] = src [ i ] [ j ] [ k ]  # if dim == 0
>
> self[ i ][index[ i ][ j ][ k ]][ k ] = src[ i ][ j ][ k ]  # if dim == 1
>
> self[ i ][ j ][index[ i ][ j ][ k ]] = src[ i ][ j ][ k ]  # if dim == 2



接下来查看一下输入和输出的形状

In [7]:
src_data.shape, target_data.shape, onehot_target.shape

(torch.Size([4898, 11]), torch.Size([4898, 1]), torch.Size([4898, 10]))

接下来对输出进行归一化操作

In [8]:
src_data_normal = (src_data - torch.mean(src_data,dim=0))/torch.sqrt(src_data.var(dim=0))
src_data_normal

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

定义训练函数

In [9]:
def training_loop(e_epchons, model, loss_fn, optimizer, train_src_data,
                  train_target_data, val_src_data, val_target_data):
    for i in range(1, e_epchons + 1):
        # 训练集处理
        train_res = model(train_src_data)
        train_loss = loss_fn(train_res, train_target_data)

        # 验证集处理
        val_res = model(val_src_data)
        val_loss = loss_fn(val_res, val_target_data)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if i == 1 or i % 100 == 0:
            print(f'Epchon: {i}  Train Loss :{train_loss}'
                  f' Val Loss :{val_loss}')

拆分数据集为训练集和验证集

In [10]:
n_samples = src_data_normal.shape[0]
n_val = int(0.2 * n_samples)
suffled_indices = torch.randperm(n_samples)
train_indices = suffled_indices[:-n_val]
val_indices = suffled_indices[-n_val:]

train_src_data = src_data_normal[train_indices, :]
train_target_data = onehot_target[train_indices, :]
val_src_data = src_data[val_indices, :]
val_target_data = onehot_target[val_indices, :]

train_target_img = target_data[train_indices,:]
val_target_img = target_data[val_indices,:]

train_src_data, train_target_data, val_src_data, val_target_data

(tensor([[-0.3019,  0.0175, -0.8609,  ..., -0.4521,  0.1766, -1.3930],
         [ 0.8831,  0.0671, -0.1173,  ..., -1.2468, -0.3491, -1.0680],
         [ 0.0536, -0.8755, -0.1999,  ..., -1.7104,  0.2642, -1.1492],
         ...,
         [ 1.5941, -0.1810,  0.8743,  ..., -0.3196,  1.1404, -0.2554],
         [-0.4204,  0.8111, -1.4394,  ...,  0.0115, -0.8749, -1.6368],
         [ 2.1866, -1.2723, -0.2825,  ...,  0.2102, -0.9625, -0.5804]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[ 6.6000,  0.2200,  0.3700,  ...,  3.3700,  0.5500, 10.3000],
         [ 6.5000,  0.4600,  0.2400,  ...,  3.0800,  0.5600,  9.8000],
         [ 6.5000,  0.2800,  0.3300,  ...,  3.2200,  0.5100,  9.7000],
         ...,
         [ 5.8000,  0.3600,  0.2600,  ...,  3.3400,  0.5500, 11.3000]

In [11]:
train_src_data.shape, train_target_data.shape, val_src_data.shape, val_target_data.shape

(torch.Size([3919, 11]),
 torch.Size([3919, 10]),
 torch.Size([979, 11]),
 torch.Size([979, 10]))

定义模型：
    
    1. 从整体数据上看输入的特征有 11 个，而输出的特征为一个分数，在这个例子中使用的独热编码把10分分到了一维向量中，因此总体而言是输入了 11 个特征输出了 10 个特征，先使用单个 `nn.Linear` 进行测试

In [12]:
linear_model = nn.Linear(11,10)
linear_model

Linear(in_features=11, out_features=10, bias=True)

In [13]:
optimizer = optim.SGD(params=linear_model.parameters(), lr=1e-2)
training_loop(e_epchons=5000,
              model=linear_model,
              optimizer=optimizer,
              loss_fn=nn.MSELoss(),
              train_target_data=train_target_data,
              train_src_data=train_src_data,
              val_target_data=val_target_data,
              val_src_data=val_src_data)

Epchon: 1  Train Loss :0.43926993012428284 Val Loss :711.9766845703125
Epchon: 100  Train Loss :0.2930648624897003 Val Loss :571.354248046875
Epchon: 200  Train Loss :0.2123730331659317 Val Loss :457.7567443847656
Epchon: 300  Train Loss :0.16375622153282166 Val Loss :367.0937805175781
Epchon: 400  Train Loss :0.1326463222503662 Val Loss :296.0086975097656
Epchon: 500  Train Loss :0.11201494932174683 Val Loss :240.7967071533203
Epchon: 600  Train Loss :0.09800571948289871 Val Loss :197.9016876220703
Epchon: 700  Train Loss :0.08831840008497238 Val Loss :164.34112548828125
Epchon: 800  Train Loss :0.08151186257600784 Val Loss :137.79823303222656
Epchon: 900  Train Loss :0.07665643841028214 Val Loss :116.54292297363281
Epchon: 1000  Train Loss :0.07314075529575348 Val Loss :99.30558013916016
Epchon: 1100  Train Loss :0.07055693119764328 Val Loss :85.15858459472656
Epchon: 1200  Train Loss :0.06862953305244446 Val Loss :73.4212875366211
Epchon: 1300  Train Loss :0.06717044860124588 Val Lo

In [14]:
optimizer.param_groups

[{'params': [Parameter containing:
   tensor([[-2.3885e-02, -4.5027e-03, -2.0844e-03, -6.7103e-02, -2.5406e-03,
            -2.1463e-04,  1.3225e-03,  1.0413e-01, -2.0044e-02, -6.3890e-03,
             5.2217e-02],
           [-4.2957e-03,  1.9599e-03, -4.8351e-04, -1.9511e-02, -4.5820e-04,
             1.0259e-02, -1.2614e-02,  3.1502e-02, -3.6593e-03, -5.1320e-04,
             1.2296e-02],
           [-3.9516e-03,  5.8391e-04, -4.2394e-04, -1.5416e-02, -4.0370e-04,
             5.0048e-03, -6.0408e-03,  2.4518e-02, -3.2661e-03, -9.3686e-04,
             1.0606e-02],
           [-1.4989e-02, -1.6429e-03, -2.5878e-03, -5.2741e-02,  4.1427e-04,
             1.4428e-03,  6.2988e-03,  7.9819e-02, -1.4691e-02, -6.6840e-03,
             4.3272e-02],
           [ 4.3082e-02,  3.4047e-02, -3.3426e-03,  6.4325e-02,  6.9212e-03,
            -2.0240e-02, -5.2955e-03, -1.1992e-01,  3.0313e-02,  4.9883e-03,
            -8.0951e-02],
           [-3.3940e-02,  7.7269e-02,  3.4240e-03, -1.2078e-01, -

然后重新定义模型，是用多层神经网络

In [15]:
linear_model = nn.Sequential(nn.Linear(11, 30), nn.Tanh(), nn.Linear(30, 10))
optimizer = optim.SGD(params=linear_model.parameters(), lr=1e-2)
training_loop(e_epchons=5000,
              model=linear_model,
              optimizer=optimizer,
              loss_fn=nn.MSELoss(),
              train_target_data=train_target_data,
              train_src_data=train_src_data,
              val_target_data=val_target_data,
              val_src_data=val_src_data)

Epchon: 1  Train Loss :0.1685260683298111 Val Loss :0.38332706689834595
Epchon: 100  Train Loss :0.12446407228708267 Val Loss :0.3292483985424042
Epchon: 200  Train Loss :0.10116351395845413 Val Loss :0.29695621132850647
Epchon: 300  Train Loss :0.08816388249397278 Val Loss :0.27459096908569336
Epchon: 400  Train Loss :0.0804595947265625 Val Loss :0.25797101855278015
Epchon: 500  Train Loss :0.0756184458732605 Val Loss :0.24513232707977295
Epchon: 600  Train Loss :0.07240107655525208 Val Loss :0.23493944108486176
Epchon: 700  Train Loss :0.07015135884284973 Val Loss :0.2266739457845688
Epchon: 800  Train Loss :0.06850852072238922 Val Loss :0.2198581099510193
Epchon: 900  Train Loss :0.06726600974798203 Val Loss :0.2141614705324173
Epchon: 1000  Train Loss :0.06630011647939682 Val Loss :0.2093481868505478
Epchon: 1100  Train Loss :0.06553322821855545 Val Loss :0.20524461567401886
Epchon: 1200  Train Loss :0.06491425633430481 Val Loss :0.20171941816806793
Epchon: 1300  Train Loss :0.0644